In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [3]:
import faiss
import pickle
import numpy as np
from langchain.schema import Document
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoModel, AutoTokenizer
import torch

def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
    # 모든 입력 텐서를 GPU로 이동
    inputs = {key: val.to(device) for key, val in inputs.items()}
    pool_mask = torch.ones(inputs["input_ids"].shape, dtype=torch.long).to(device)

    with torch.no_grad():
        outputs = model(**inputs, pool_mask=pool_mask)

    return outputs.embedding.cpu().numpy()  # GPU에서 CPU로 변환 후 반환

model_name = "kakaocorp/kanana-nano-2.1b-embedding"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


c:\Users\USER\anaconda3\envs\vectordb_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# 1️⃣ 저장된 FAISS 인덱스 로드
index = faiss.read_index("./faiss_index.bin")

# 2️⃣ 문서 정보 로드
with open("./documents.pkl", "rb") as f:
    documents = pickle.load(f)

# 3️⃣ 검색 수행
# query = " 보험금의 종류 및 한도에 대해 설명해줘?"
query = "자동차 의무보험 미가입에 따른 불이익을 알려줘"
query_embedding = get_embeddings([query])[0]  # 쿼리 임베딩
query_embedding = np.array([query_embedding], dtype=np.float32)

D, I = index.search(query_embedding, k=10)  # 가장 유사한 5개 검색
context = []

# 4️⃣ 검색 결과 출력
for idx in I[0]:
    context.append(documents[idx].page_content)
    print(f"🔹 문서 {idx}: {documents[idx].page_content[:200]}")


🔹 문서 8: 보 통 약 관 특 별 약 관 교 통 사 고 발 생 시 대 처 요 령 관 련 법 령 알 기 쉬 운 자 동 차 보 험 이 야 기 개인용자동차보험 9 1 청약내용 확인 관련 보험기간 중 계약 관련 사항이 변경된 경우 계약 담당자나  고객 콜센터(1588-3344, 1600-3434)로 지체없이 연락 주시기 바랍니다. • 고객 주소지 또는 연락처가 변경된 경우 고
🔹 문서 24: 보 통 약 관 특 별 약 관 교 통 사 고 발 생 시 대 처 요 령 관 련 법 령 알 기 쉬 운 자 동 차 보 험 이 야 기 개인용자동차보험 25 Q 자기차량손해담보 가입금액은 어떻게 정해지나요? Q 운전 범위에 포함되지 않는 사람이 일시적으로 운전할 수 있는 방법이 있나요? A 단기운전자 확대 특약을 통해 일시적으로 운전자 범위를 확대할 수 있습니다. 계
🔹 문서 22: 보 통 약 관 특 별 약 관 교 통 사 고 발 생 시 대 처 요 령 관 련 법 령 알 기 쉬 운 자 동 차 보 험 이 야 기 개인용자동차보험 23 구분 주요내용 차선이탈방지장치 장착할인 특약 - 가입대상 : 차선이탈경고장치(LDWS) 또는 차선유지지원장치(LKAS)가 장착된 차량 - 제출서류 : 출고시 기본 장착된 차량일 경우 특약자동가입, 옵션 장착의 경
🔹 문서 18: 보 통 약 관 특 별 약 관 교 통 사 고 발 생 시 대 처 요 령 관 련 법 령 알 기 쉬 운 자 동 차 보 험 이 야 기 개인용자동차보험 19 ※ 유의사항 1) 배우자 : 법률상 또는 사실혼 관계의 배우자 2) 가족의 범위에 포함되지 않는 사람 : 형제·자매·남매, (외)조부모, 손자·녀, 동거인, 고용인 등 3) 지정1인 : 가족한정 특약의 가족 범위
🔹 문서 6: 보 통 약 관 특 별 약 관 교 통 사 고 발 생 시 대 처 요 령 관 련 법 령 알 기 쉬 운 자 동 차 보 험 이 야 기 개인용자동차보험 7 1 의무보험의 의미 ‘의무보험’은 자동차 사고 피해자가 최소한의 보상을 받을 수 있도록 자동차손해 배상보장법에 의거하여 국가에서 

In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Create a FAISS VectorStore
vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings() # deployment="text-embedding-ada-002"
)

# Create a Retriever
retriever = vectorstore.as_retriever()

system_message = "당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요."
prompt_text = f"[SYSTEM]: {system_message}\n[USER]: {query}\n[context]: {context}\n[ASSISTANT]"

prompt = ChatPromptTemplate.from_template(prompt_text)
model = ChatOpenAI() # model_name='gpt-3.5-turbo'

# Chaining a pipeline
retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Run retrieva_chain
retrieval_chain.invoke("where did harrison work?")


C:\Users\USER\AppData\Local\Temp\ipykernel_14392\3628069505.py:20: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI() # model_name='gpt-3.5-turbo'


'자동차 보험의 경우 의무보험 미가입에 따른 불이익은 다음과 같습니다. 의무보험 미가입 상황에서 운행 시 1년 이하의 징역 또는 1천만원 이하의 벌금이 부과됩니다. 의무보험 미가입 기간에 따른 과태료는 다음과 같습니다.\n\n- 담보 미가입시 과태료\n  - 10일 이내: 대인배상 I - 1만원, 대물배상 - 5천원\n  - 10일 초과시: 대인배상 I - 4천원, 대물배상 - 2천원\n\n의무보험은 국가에서 법적으로 가입을 의무화한 보험으로, 자동차 사고 피해자가 최소한의 보상을 받을 수 있도록 설계되어 있습니다. 의무보험 미가입 시 법적인 제재가 가해지므로, 정확한 보험 가입 여부를 확인하고 가입에 만전을 기하시는 것이 중요합니다. 부가적인 담보에 가입하여 보다 안전한 보험 보호를 받을 수 있습니다.'

In [ ]:
import torch
from transformers import PhiForCausalLM, AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cpu")

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained("daekeun-ml/phi-2-ko-v0.1", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained("daekeun-ml/phi-2-ko-v0.1", trust_remote_code=True)

system_message = "당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요."
prompt = f"[SYSTEM]: {system_message}\n[USER]: {query}\n[context]: {context}\n[ASSISTANT]"

inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_new_tokens=512)
text = tokenizer.batch_decode(outputs)[0]
print(text) # 12분

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.70it/s]
Token indices sequence length is longer than the specified maximum sequence length for this model (4858 > 2048). Running this sequence through the model will result in indexing errors
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


[SYSTEM]: 당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요.
[USER]: 자동차 의무보험 미가입에 따른 불이익을 알려줘
[context]: ['보 통 약 관 특 별 약 관 교 통 사 고 발 생 시 대 처 요 령 관 련 법 령 알 기 쉬 운 자 동 차 보 험 이 야 기 개인용자동차보험 9 1 청약내용 확인 관련 보험기간 중 계약 관련 사항이 변경된 경우 계약 담당자나  고객 콜센터(1588-3344, 1600-3434)로 지체없이 연락 주시기 바랍니다. • 고객 주소지 또는 연락처가 변경된 경우 고객님께 발송되는 각종 안내문(만기안내장 등) 수령을 위하여 변경된 주소와 연락처를 알려주시기 바랍니다. • 운전가능자의 범위가 변경된 경우 피보험자동차를 운전하려는 자의 범위 또는 연령이 변경된 경우에는 운전자한정/연령한정 특약을 변경하셔야 합니다. • 새차를 구매하였거나 교체한 경우 새로운 차를 구매하여 기존 차량과 대체한 경우 차량변경 사실을 보험사에 알리고 승인을 받으셔야 합니다. • 새로운 차량 부속품이 추가된 경우 자동차에 장착된 기본 사양 이외에 새로운 부속품이 추가된 경우, 부속품 종류와 가액을 보험회사에 알려주셔야 합니다. 4. 가입자 유의사항 : 보험계약 만료 시 현행 자동차손해배상보장법 제6조 제1항의 규정에 의거, 자동차보험을 가입한 고객님께 보험계약 만료(종료) 전 2회 에 걸쳐 만료사실을 안내하고 있습니다. 구분 안내시기 안내방법 안내대상 1차 안내 만료일 75일~30일전 모바일/이메일/우편 계약자 2차 안내 만료일 30일~10일전 우편 계약자 ※ 보험기간이 1개월 이내인 단기계약의 경우에는 안내가 생략될 수 있습니다. 3. 가입자 유의사항 : 보험계약 변경 시', '보 통 약 관 특 별 약 관 교 통 사 고 발 생 시 대 처 요 령 관 련 법 령 알 기 쉬 운 자 동 차 보 험 이 야 기 개인용자동차보험 25 Q 자기차량손해담보 가입금액은 어떻게 정해지나요? Q 운전 범위에

In [ ]:
import torch
from transformers import PhiForCausalLM, AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cpu")

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained("kakaocorp/kanana-nano-2.1b-base", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained("kakaocorp/kanana-nano-2.1b-base", trust_remote_code=True)

system_message = "당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요."
prompt = f"[SYSTEM]: {system_message}\n[USER]: {query}\n[context]: {context}\n[ASSISTANT]"

inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_new_tokens=512, max_length=2048)
text = tokenizer.batch_decode(outputs)[0]
print(text) # 14분

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


<|begin_of_text|>[SYSTEM]: 당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요.
[USER]: 자동차 의무보험 미가입에 따른 불이익을 알려줘
[context]: ['보 통 약 관 특 별 약 관 교 통 사 고 발 생 시 대 처 요 령 관 련 법 령 알 기 쉬 운 자 동 차 보 험 이 야 기 개인용자동차보험 9 1 청약내용 확인 관련 보험기간 중 계약 관련 사항이 변경된 경우 계약 담당자나  고객 콜센터(1588-3344, 1600-3434)로 지체없이 연락 주시기 바랍니다. • 고객 주소지 또는 연락처가 변경된 경우 고객님께 발송되는 각종 안내문(만기안내장 등) 수령을 위하여 변경된 주소와 연락처를 알려주시기 바랍니다. • 운전가능자의 범위가 변경된 경우 피보험자동차를 운전하려는 자의 범위 또는 연령이 변경된 경우에는 운전자한정/연령한정 특약을 변경하셔야 합니다. • 새차를 구매하였거나 교체한 경우 새로운 차를 구매하여 기존 차량과 대체한 경우 차량변경 사실을 보험사에 알리고 승인을 받으셔야 합니다. • 새로운 차량 부속품이 추가된 경우 자동차에 장착된 기본 사양 이외에 새로운 부속품이 추가된 경우, 부속품 종류와 가액을 보험회사에 알려주셔야 합니다. 4. 가입자 유의사항 : 보험계약 만료 시 현행 자동차손해배상보장법 제6조 제1항의 규정에 의거, 자동차보험을 가입한 고객님께 보험계약 만료(종료) 전 2회 에 걸쳐 만료사실을 안내하고 있습니다. 구분 안내시기 안내방법 안내대상 1차 안내 만료일 75일~30일전 모바일/이메일/우편 계약자 2차 안내 만료일 30일~10일전 우편 계약자 ※ 보험기간이 1개월 이내인 단기계약의 경우에는 안내가 생략될 수 있습니다. 3. 가입자 유의사항 : 보험계약 변경 시', '보 통 약 관 특 별 약 관 교 통 사 고 발 생 시 대 처 요 령 관 련 법 령 알 기 쉬 운 자 동 차 보 험 이 야 기 개인용자동차보험 25 Q 자기차량손해담보 가입금액은 어떻

In [25]:
import torch
from transformers import PhiForCausalLM, AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cpu")

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained("upskyy/e5-small-korean", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained("upskyy/e5-small-korean", trust_remote_code=True)

system_message = "당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요."
prompt = f"[SYSTEM]: {system_message}\n[USER]: {query}\n[context]: {context}\n[ASSISTANT]"

inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_new_tokens=512, max_length=2048)
text = tokenizer.batch_decode(outputs)[0]
print(text)

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of BertLMHeadModel were not initialized from the model checkpoint at upskyy/e5-small-korean and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Token indices sequence length is longer than the specified maximum sequence length for this model (5065 > 512). Running this sequence through the model will result in indexing errors
Both `max_new_tokens` (=512) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


RuntimeError: The expanded size of the tensor (5065) must match the existing size (512) at non-singleton dimension 1.  Target sizes: [1, 5065].  Tensor sizes: [1, 512]

In [ ]:
import torch
from transformers import PhiForCausalLM, AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cpu")

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-4-mini-instruct", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-4-mini-instruct", trust_remote_code=True)

system_message = "당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요."
prompt = f"[SYSTEM]: {system_message}\n[USER]: {query}\n[context]: {context}\n[ASSISTANT]"

inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_new_tokens=512, max_length=2048)
text = tokenizer.batch_decode(outputs)[0]
print(text) # 18분

c:\Users\USER\anaconda3\envs\vectordb_env\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--microsoft--Phi-4-mini-instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  5.07it/s]
Both `max_new_tokens` (=5

[SYSTEM]: 당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요.
[USER]: 자동차 의무보험 미가입에 따른 불이익을 알려줘
[context]: ['보 통 약 관 특 별 약 관 교 통 사 고 발 생 시 대 처 요 령 관 련 법 령 알 기 쉬 운 자 동 차 보 험 이 야 기 개인용자동차보험 9 1 청약내용 확인 관련 보험기간 중 계약 관련 사항이 변경된 경우 계약 담당자나  고객 콜센터(1588-3344, 1600-3434)로 지체없이 연락 주시기 바랍니다. • 고객 주소지 또는 연락처가 변경된 경우 고객님께 발송되는 각종 안내문(만기안내장 등) 수령을 위하여 변경된 주소와 연락처를 알려주시기 바랍니다. • 운전가능자의 범위가 변경된 경우 피보험자동차를 운전하려는 자의 범위 또는 연령이 변경된 경우에는 운전자한정/연령한정 특약을 변경하셔야 합니다. • 새차를 구매하였거나 교체한 경우 새로운 차를 구매하여 기존 차량과 대체한 경우 차량변경 사실을 보험사에 알리고 승인을 받으셔야 합니다. • 새로운 차량 부속품이 추가된 경우 자동차에 장착된 기본 사양 이외에 새로운 부속품이 추가된 경우, 부속품 종류와 가액을 보험회사에 알려주셔야 합니다. 4. 가입자 유의사항 : 보험계약 만료 시 현행 자동차손해배상보장법 제6조 제1항의 규정에 의거, 자동차보험을 가입한 고객님께 보험계약 만료(종료) 전 2회 에 걸쳐 만료사실을 안내하고 있습니다. 구분 안내시기 안내방법 안내대상 1차 안내 만료일 75일~30일전 모바일/이메일/우편 계약자 2차 안내 만료일 30일~10일전 우편 계약자 ※ 보험기간이 1개월 이내인 단기계약의 경우에는 안내가 생략될 수 있습니다. 3. 가입자 유의사항 : 보험계약 변경 시', '보 통 약 관 특 별 약 관 교 통 사 고 발 생 시 대 처 요 령 관 련 법 령 알 기 쉬 운 자 동 차 보 험 이 야 기 개인용자동차보험 25 Q 자기차량손해담보 가입금액은 어떻게 정해지나요? Q 운전 범위에

In [ ]:
import torch
from transformers import PhiForCausalLM, AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cpu")

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained("Bllossom/llama-3.2-Korean-Bllossom-3B", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained("Bllossom/llama-3.2-Korean-Bllossom-3B", trust_remote_code=True)

system_message = "당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요."
prompt = f"[SYSTEM]: {system_message}\n[USER]: {query}\n[context]: {context}\n[ASSISTANT]"

inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_new_tokens=512, max_length=2048)
text = tokenizer.batch_decode(outputs)[0]

c:\Users\USER\anaconda3\envs\vectordb_env\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--Bllossom--llama-3.2-Korean-Bllossom-3B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  4.28it/s]
The attention mas

In [ ]:
print(text) # 16분

<|begin_of_text|>[SYSTEM]: 당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요.
[USER]: 자동차 의무보험 미가입에 따른 불이익을 알려줘
[context]: ['보 통 약 관 특 별 약 관 교 통 사 고 발 생 시 대 처 요 령 관 련 법 령 알 기 쉬 운 자 동 차 보 험 이 야 기 개인용자동차보험 9 1 청약내용 확인 관련 보험기간 중 계약 관련 사항이 변경된 경우 계약 담당자나  고객 콜센터(1588-3344, 1600-3434)로 지체없이 연락 주시기 바랍니다. • 고객 주소지 또는 연락처가 변경된 경우 고객님께 발송되는 각종 안내문(만기안내장 등) 수령을 위하여 변경된 주소와 연락처를 알려주시기 바랍니다. • 운전가능자의 범위가 변경된 경우 피보험자동차를 운전하려는 자의 범위 또는 연령이 변경된 경우에는 운전자한정/연령한정 특약을 변경하셔야 합니다. • 새차를 구매하였거나 교체한 경우 새로운 차를 구매하여 기존 차량과 대체한 경우 차량변경 사실을 보험사에 알리고 승인을 받으셔야 합니다. • 새로운 차량 부속품이 추가된 경우 자동차에 장착된 기본 사양 이외에 새로운 부속품이 추가된 경우, 부속품 종류와 가액을 보험회사에 알려주셔야 합니다. 4. 가입자 유의사항 : 보험계약 만료 시 현행 자동차손해배상보장법 제6조 제1항의 규정에 의거, 자동차보험을 가입한 고객님께 보험계약 만료(종료) 전 2회 에 걸쳐 만료사실을 안내하고 있습니다. 구분 안내시기 안내방법 안내대상 1차 안내 만료일 75일~30일전 모바일/이메일/우편 계약자 2차 안내 만료일 30일~10일전 우편 계약자 ※ 보험기간이 1개월 이내인 단기계약의 경우에는 안내가 생략될 수 있습니다. 3. 가입자 유의사항 : 보험계약 변경 시', '보 통 약 관 특 별 약 관 교 통 사 고 발 생 시 대 처 요 령 관 련 법 령 알 기 쉬 운 자 동 차 보 험 이 야 기 개인용자동차보험 25 Q 자기차량손해담보 가입금액은 어떻

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
from threading import Thread

model_name = "LGAI-EXAONE/EXAONE-Deep-2.4B"
streaming = True    # choose the streaming option

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

#   Korean MCQA example (CSAT Math 2025)
prompt = f"""당신은 보험 상담 ASSISTANT입니다. 보험 약관에서 추출한 context의 내용을 기반으로 USER의 질문인 query에 대해 한국어로 간결하게 답변해 주세요.
[USER]: {query}
[context]: {context}
[ASSISTANT]:"""

messages = [
    {"role": "user", "content": prompt}
]
input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)

if streaming:
    streamer = TextIteratorStreamer(tokenizer)
    thread = Thread(target=model.generate, kwargs=dict(
        input_ids=input_ids.to("cpu"),
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=32768,
        do_sample=True,
        temperature=0.6,
        top_p=0.95,
        streamer=streamer
    ))
    thread.start()

    for text in streamer:
        print(text, end="", flush=True)
else:
    output = model.generate(
        input_ids.to("cpu"),
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=32768,
        do_sample=True,
        temperature=0.6,
        top_p=0.95,
    )
    print(tokenizer.decode(output[0])) # 20분


[|system|][|endofturn|]
[|user|]당신은 보험 상담 ASSISTANT입니다. 보험 약관에서 추출한 context의 내용을 기반으로 USER의 질문인 query에 대해 한국어로 간결하게 답변해 주세요.
[USER]: 자동차 의무보험 미가입에 따른 불이익을 알려줘
[context]: ['보 통 약 관 특 별 약 관 교 통 사 고 발 생 시 대 처 요 령 관 련 법 령 알 기 쉬 운 자 동 차 보 험 이 야 기 개인용자동차보험 9 1 청약내용 확인 관련 보험기간 중 계약 관련 사항이 변경된 경우 계약 담당자나  고객 콜센터(1588-3344, 1600-3434)로 지체없이 연락 주시기 바랍니다. • 고객 주소지 또는 연락처가 변경된 경우 고객님께 발송되는 각종 안내문(만기안내장 등) 수령을 위하여 변경된 주소와 연락처를 알려주시기 바랍니다. • 운전가능자의 범위가 변경된 경우 피보험자동차를 운전하려는 자의 범위 또는 연령이 변경된 경우에는 운전자한정/연령한정 특약을 변경하셔야 합니다. • 새차를 구매하였거나 교체한 경우 새로운 차를 구매하여 기존 차량과 대체한 경우 차량변경 사실을 보험사에 알리고 승인을 받으셔야 합니다. • 새로운 차량 부속품이 추가된 경우 자동차에 장착된 기본 사양 이외에 새로운 부속품이 추가된 경우, 부속품 종류와 가액을 보험회사에 알려주셔야 합니다. 4. 가입자 유의사항 : 보험계약 만료 시 현행 자동차손해배상보장법 제6조 제1항의 규정에 의거, 자동차보험을 가입한 고객님께 보험계약 만료(종료) 전 2회 에 걸쳐 만료사실을 안내하고 있습니다. 구분 안내시기 안내방법 안내대상 1차 안내 만료일 75일~30일전 모바일/이메일/우편 계약자 2차 안내 만료일 30일~10일전 우편 계약자 ※ 보험기간이 1개월 이내인 단기계약의 경우에는 안내가 생략될 수 있습니다. 3. 가입자 유의사항 : 보험계약 변경 시', '보 통 약 관 특 별 약 관 교 통 사 고 발 생 시 대 처 요 령 관 련 법 령 알 기 쉬 운 자 동 차 보 험 이 

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
from threading import Thread

model_name = "LGAI-EXAONE/EXAONE-Deep-2.4B"
streaming = True    # choose the streaming option

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

#   Korean MCQA example (CSAT Math 2025)
prompt = f"""당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요.
[SYSTEM]: {system_message}\n[USER]: {query}\n[context]: {context}\n[ASSISTANT]"""

messages = [
    {"role": "user", "content": prompt}
]
input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)

if streaming:
    streamer = TextIteratorStreamer(tokenizer)
    thread = Thread(target=model.generate, kwargs=dict(
        input_ids=input_ids.to("cpu"),
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=32768,
        do_sample=True,
        temperature=0.6,
        top_p=0.95,
        streamer=streamer
    ))
    thread.start()

    for text in streamer:
        print(text, end="", flush=True)
else:
    output = model.generate(
        input_ids.to("cpu"),
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=32768,
        do_sample=True,
        temperature=0.6,
        top_p=0.95,
    )
    print(tokenizer.decode(output[0]))# 21분


[|system|][|endofturn|]
[|user|]당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요.
[SYSTEM]: 당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요.
[USER]: 자동차 의무보험 미가입에 따른 불이익을 알려줘
[context]: ['보 통 약 관 특 별 약 관 교 통 사 고 발 생 시 대 처 요 령 관 련 법 령 알 기 쉬 운 자 동 차 보 험 이 야 기 개인용자동차보험 9 1 청약내용 확인 관련 보험기간 중 계약 관련 사항이 변경된 경우 계약 담당자나  고객 콜센터(1588-3344, 1600-3434)로 지체없이 연락 주시기 바랍니다. • 고객 주소지 또는 연락처가 변경된 경우 고객님께 발송되는 각종 안내문(만기안내장 등) 수령을 위하여 변경된 주소와 연락처를 알려주시기 바랍니다. • 운전가능자의 범위가 변경된 경우 피보험자동차를 운전하려는 자의 범위 또는 연령이 변경된 경우에는 운전자한정/연령한정 특약을 변경하셔야 합니다. • 새차를 구매하였거나 교체한 경우 새로운 차를 구매하여 기존 차량과 대체한 경우 차량변경 사실을 보험사에 알리고 승인을 받으셔야 합니다. • 새로운 차량 부속품이 추가된 경우 자동차에 장착된 기본 사양 이외에 새로운 부속품이 추가된 경우, 부속품 종류와 가액을 보험회사에 알려주셔야 합니다. 4. 가입자 유의사항 : 보험계약 만료 시 현행 자동차손해배상보장법 제6조 제1항의 규정에 의거, 자동차보험을 가입한 고객님께 보험계약 만료(종료) 전 2회 에 걸쳐 만료사실을 안내하고 있습니다. 구분 안내시기 안내방법 안내대상 1차 안내 만료일 75일~30일전 모바일/이메일/우편 계약자 2차 안내 만료일 30일~10일전 우편 계약자 ※ 보험기간이 1개월 이내인 단기계약의 경우에는 안내가 생략될 수 있습니다. 3. 가입자 유의사항 : 보험계약 변경 시', '보 통 약 관 특 별 약 관 교 통 사 고

In [12]:
import torch
from transformers import PhiForCausalLM, AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cpu")

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained("skt/ko-gpt-trinity-1.2B-v0.5", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained("skt/ko-gpt-trinity-1.2B-v0.5", trust_remote_code=True, return_attention_mask=False, truncation=True, max_length=1024)

system_message = "당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요."
prompt = f"[SYSTEM]: {system_message}\n[USER]: {query}\n[context]: {context}\n[ASSISTANT]"

inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_new_tokens=100, max_length=1024)
text = tokenizer.batch_decode(outputs)[0]
print(text) 

Both `max_new_tokens` (=100) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


IndexError: index out of range in self

In [6]:
import torch
from transformers import PhiForCausalLM, AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cpu")

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained("EleutherAI/polyglot-ko-1.3b", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/polyglot-ko-1.3b", trust_remote_code=True)

system_message = "당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요."
prompt = f"[SYSTEM]: {system_message}\n[USER]: {query}\n[context]: {context}\n[ASSISTANT]"

inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_new_tokens=512, max_length=2048)
text = tokenizer.batch_decode(outputs)[0]
print(text) # 16분

c:\Users\USER\anaconda3\envs\vectordb_env\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--EleutherAI--polyglot-ko-1.3b. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  3.26it/s]


ValueError: The following `model_kwargs` are not used by the model: ['token_type_ids'] (note: typos in the generate arguments will also show up in this list)

In [9]:
import torch
from transformers import PhiForCausalLM, AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cpu")

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained("quantumaikr/KoreanLM-3B", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained("quantumaikr/KoreanLM-3B", trust_remote_code=True)

system_message = "당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요."
prompt = f"[SYSTEM]: {system_message}\n[USER]: {query}\n[context]: {context}\n[ASSISTANT]"

inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_new_tokens=512, max_length=2048)
text = tokenizer.batch_decode(outputs)[0]
print(text) # 16분

Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  3.93it/s]
Both `max_new_tokens` (=512) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[SYSTEM]: 당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요.
[USER]: 자동차 의무보험 미가입에 따른 불이익을 알려줘
[context]: ['보 통 약 관 특 별 약 관 교 통 사 고 발 생 시 대 처 요 령 관 련 법 령 알 기 쉬 운 자 동 차 보 험 이 야 기 개인용자동차보험 9 1 청약내용 확인 관련 보험기간 중 계약 관련 사항이 변경된 경우 계약 담당자나  고객 콜센터(1588-3344, 1600-3434)로 지체없이 연락 주시기 바랍니다. • 고객 주소지 또는 연락처가 변경된 경우 고객님께 발송되는 각종 안내문(만기안내장 등) 수령을 위하여 변경된 주소와 연락처를 알려주시기 바랍니다. • 운전가능자의 범위가 변경된 경우 피보험자동차를 운전하려는 자의 범위 또는 연령이 변경된 경우에는 운전자한정/연령한정 특약을 변경하셔야 합니다. • 새차를 구매하였거나 교체한 경우 새로운 차를 구매하여 기존 차량과 대체한 경우 차량변경 사실을 보험사에 알리고 승인을 받으셔야 합니다. • 새로운 차량 부속품이 추가된 경우 자동차에 장착된 기본 사양 이외에 새로운 부속품이 추가된 경우, 부속품 종류와 가액을 보험회사에 알려주셔야 합니다. 4. 가입자 유의사항 : 보험계약 만료 시 현행 자동차손해배상보장법 제6조 제1항의 규정에 의거, 자동차보험을 가입한 고객님께 보험계약 만료(종료) 전 2회 에 걸쳐 만료사실을 안내하고 있습니다. 구분 안내시기 안내방법 안내대상 1차 안내 만료일 75일~30일전 모바일/이메일/우편 계약자 2차 안내 만료일 30일~10일전 우편 계약자 ※ 보험기간이 1개월 이내인 단기계약의 경우에는 안내가 생략될 수 있습니다. 3. 가입자 유의사항 : 보험계약 변경 시', '보 통 약 관 특 별 약 관 교 통 사 고 발 생 시 대 처 요 령 관 련 법 령 알 기 쉬 운 자 동 차 보 험 이 야 기 개인용자동차보험 25 Q 자기차량손해담보 가입금액은 어떻게 정해지나요? Q 운전 범위에

In [ ]:
import torch
from transformers import PhiForCausalLM, AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cpu")

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained("beomi/gemma-ko-2b", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained("beomi/gemma-ko-2b", trust_remote_code=True)

system_message = "당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요."
prompt = f"[SYSTEM]: {system_message}\n[USER]: {query}\n[context]: {context}\n[ASSISTANT]"

inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_new_tokens=512, max_length=2048)
text = tokenizer.batch_decode(outputs)[0]
print(text) # 14분

c:\Users\USER\anaconda3\envs\vectordb_env\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--beomi--gemma-ko-2b. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.03s/it]
Both `max_new_tokens` (=512) and `max

<bos>[SYSTEM]: 당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요.
[USER]: 자동차 의무보험 미가입에 따른 불이익을 알려줘
[context]: ['보 통 약 관 특 별 약 관 교 통 사 고 발 생 시 대 처 요 령 관 련 법 령 알 기 쉬 운 자 동 차 보 험 이 야 기 개인용자동차보험 9 1 청약내용 확인 관련 보험기간 중 계약 관련 사항이 변경된 경우 계약 담당자나  고객 콜센터(1588-3344, 1600-3434)로 지체없이 연락 주시기 바랍니다. • 고객 주소지 또는 연락처가 변경된 경우 고객님께 발송되는 각종 안내문(만기안내장 등) 수령을 위하여 변경된 주소와 연락처를 알려주시기 바랍니다. • 운전가능자의 범위가 변경된 경우 피보험자동차를 운전하려는 자의 범위 또는 연령이 변경된 경우에는 운전자한정/연령한정 특약을 변경하셔야 합니다. • 새차를 구매하였거나 교체한 경우 새로운 차를 구매하여 기존 차량과 대체한 경우 차량변경 사실을 보험사에 알리고 승인을 받으셔야 합니다. • 새로운 차량 부속품이 추가된 경우 자동차에 장착된 기본 사양 이외에 새로운 부속품이 추가된 경우, 부속품 종류와 가액을 보험회사에 알려주셔야 합니다. 4. 가입자 유의사항 : 보험계약 만료 시 현행 자동차손해배상보장법 제6조 제1항의 규정에 의거, 자동차보험을 가입한 고객님께 보험계약 만료(종료) 전 2회 에 걸쳐 만료사실을 안내하고 있습니다. 구분 안내시기 안내방법 안내대상 1차 안내 만료일 75일~30일전 모바일/이메일/우편 계약자 2차 안내 만료일 30일~10일전 우편 계약자 ※ 보험기간이 1개월 이내인 단기계약의 경우에는 안내가 생략될 수 있습니다. 3. 가입자 유의사항 : 보험계약 변경 시', '보 통 약 관 특 별 약 관 교 통 사 고 발 생 시 대 처 요 령 관 련 법 령 알 기 쉬 운 자 동 차 보 험 이 야 기 개인용자동차보험 25 Q 자기차량손해담보 가입금액은 어떻게 정해지나요? Q 운

In [15]:
import torch
from transformers import PhiForCausalLM, AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cpu")

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-4-mini-instruct-onnx", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-4-mini-instruct-onnx", trust_remote_code=True)

system_message = "당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요."
prompt = f"[SYSTEM]: {system_message}\n[USER]: {query}\n[context]: {context}\n[ASSISTANT]"

inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_new_tokens=512, max_length=2048)
text = tokenizer.batch_decode(outputs)[0]
print(text) # 16분

c:\Users\USER\anaconda3\envs\vectordb_env\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--microsoft--Phi-4-mini-instruct-onnx. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


OSError: microsoft/Phi-4-mini-instruct-onnx does not appear to have a file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt or flax_model.msgpack.

In [16]:
!huggingface-cli download microsoft/Phi-4-mini-instruct-onnx --include cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4/* --local-dir .

C:\KIMUJUNG\team_project\team_project3



Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]Still waiting to acquire lock on .cache\huggingface\.gitignore.lock (elapsed: 0.1 seconds)
Still waiting to acquire lock on .cache\huggingface\.gitignore.lock (elapsed: 0.1 seconds)
Download complete. Moving file to cpu_and_mobile\cpu-int4-rtn-block-32-acc-level-4\special_tokens_map.json
Download complete. Moving file to cpu_and_mobile\cpu-int4-rtn-block-32-acc-level-4\added_tokens.json

Fetching 11 files:   9%|▉         | 1/11 [00:00<00:03,  3.07it/s]Download complete. Moving file to cpu_and_mobile\cpu-int4-rtn-block-32-acc-level-4\config.json
Download complete. Moving file to cpu_and_mobile\cpu-int4-rtn-block-32-acc-level-4\genai_config.json
Download complete. Moving file to cpu_and_mobile\cpu-int4-rtn-block-32-acc-level-4\configuration_phi3.py

Fetching 11 files:  27%|██▋       | 3/11 [00:00<00:01,  5.56it/s]Download complete. Moving file to cpu_and_mobile\cpu-int4-rtn-block-32-acc-level-4\tokenizer_config.json
Download compl

In [17]:
!pip install --pre onnxruntime-genai

   ---------------------------------------- 0.0/981.6 kB ? eta -:--:--
   --------------------------------------- 981.6/981.6 kB 45.0 MB/s eta 0:00:00


In [18]:
!curl https://raw.githubusercontent.com/microsoft/onnxruntime-genai/main/examples/python/phi3-qa.py -o phi3-qa.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  4659  100  4659    0     0   8565      0 --:--:-- --:--:-- --:--:--  8595


In [23]:
!python phi3-qa.py -m cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4 -e cpu

Traceback (most recent call last):
  File "c:\KIMUJUNG\team_project\team_project3\phi3-qa.py", line 100, in <module>
    main(args)
  File "c:\KIMUJUNG\team_project\team_project3\phi3-qa.py", line 14, in main
    config.clear_providers()
AttributeError: 'onnxruntime.capi.onnxruntime_pybind11_state.Sessio' object has no attribute 'clear_providers'


In [22]:
import onnxruntime_genai
print(onnxruntime_genai.__version__)

0.4.0


In [20]:
import onnxruntime as ort
config = ort.SessionOptions()

In [21]:
import onnxruntime_genai as og
print(dir(og))

['DeviceArray_float', 'DeviceArray_int32', 'Generator', 'GeneratorParams', 'Images', 'Model', 'MultiModalProcessor', 'NamedTensors', 'OrtException', 'Tokenizer', 'TokenizerStream', '__builtins__', '__cached__', '__doc__', '__file__', '__id__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', '_dll_directory', 'get_current_gpu_device_id', 'is_cuda_available', 'is_dml_available', 'is_rocm_available', 'onnxruntime_genai', 'set_current_gpu_device_id', 'set_log_options']


In [ ]:
import torch
from transformers import PhiForCausalLM, AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cpu")

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained("42dot/42dot_LLM-PLM-1.3B", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained("42dot/42dot_LLM-PLM-1.3B", trust_remote_code=True)

system_message = "당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요."
prompt = f"[SYSTEM]: {system_message}\n[USER]: {query}\n[context]: {context}\n[ASSISTANT]"

inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_new_tokens=512, max_length=2048)
text = tokenizer.batch_decode(outputs)[0]
print(text) # 10분

c:\Users\USER\anaconda3\envs\vectordb_env\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--42dot--42dot_LLM-PLM-1.3B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Both `max_new_tokens` (=512) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take 

<||bos||>[SYSTEM]: 당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요.
[USER]: 자동차 의무보험 미가입에 따른 불이익을 알려줘
[context]: ['보 통 약 관 특 별 약 관 교 통 사 고 발 생 시 대 처 요 령 관 련 법 령 알 기 쉬 운 자 동 차 보 험 이 야 기 개인용자동차보험 9 1 청약내용 확인 관련 보험기간 중 계약 관련 사항이 변경된 경우 계약 담당자나  고객 콜센터(1588-3344, 1600-3434)로 지체없이 연락 주시기 바랍니다. • 고객 주소지 또는 연락처가 변경된 경우 고객님께 발송되는 각종 안내문(만기안내장 등) 수령을 위하여 변경된 주소와 연락처를 알려주시기 바랍니다. • 운전가능자의 범위가 변경된 경우 피보험자동차를 운전하려는 자의 범위 또는 연령이 변경된 경우에는 운전자한정/연령한정 특약을 변경하셔야 합니다. • 새차를 구매하였거나 교체한 경우 새로운 차를 구매하여 기존 차량과 대체한 경우 차량변경 사실을 보험사에 알리고 승인을 받으셔야 합니다. • 새로운 차량 부속품이 추가된 경우 자동차에 장착된 기본 사양 이외에 새로운 부속품이 추가된 경우, 부속품 종류와 가액을 보험회사에 알려주셔야 합니다. 4. 가입자 유의사항 : 보험계약 만료 시 현행 자동차손해배상보장법 제6조 제1항의 규정에 의거, 자동차보험을 가입한 고객님께 보험계약 만료(종료) 전 2회 에 걸쳐 만료사실을 안내하고 있습니다. 구분 안내시기 안내방법 안내대상 1차 안내 만료일 75일~30일전 모바일/이메일/우편 계약자 2차 안내 만료일 30일~10일전 우편 계약자 ※ 보험기간이 1개월 이내인 단기계약의 경우에는 안내가 생략될 수 있습니다. 3. 가입자 유의사항 : 보험계약 변경 시', '보 통 약 관 특 별 약 관 교 통 사 고 발 생 시 대 처 요 령 관 련 법 령 알 기 쉬 운 자 동 차 보 험 이 야 기 개인용자동차보험 25 Q 자기차량손해담보 가입금액은 어떻게 정해지나요?

In [25]:
import torch
from transformers import PhiForCausalLM, AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cpu")

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained("kuotient/mamba-ko-2.8b", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained("kuotient/mamba-ko-2.8b", trust_remote_code=True)

system_message = "당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요."
prompt = f"[SYSTEM]: {system_message}\n[USER]: {query}\n[context]: {context}\n[ASSISTANT]"

inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_new_tokens=512, max_length=2048)
text = tokenizer.batch_decode(outputs)[0]
print(text) # 16분

c:\Users\USER\anaconda3\envs\vectordb_env\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--kuotient--mamba-ko-2.8b. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
The fast path is not available because one of `(selective_state_update, selective_scan_fn, causal_conv1d_

RuntimeError: Error(s) in loading state_dict for MambaForCausalLM:
	size mismatch for backbone.embeddings.weight: copying a param with shape torch.Size([50280, 2560]) from checkpoint, the shape in current model is torch.Size([50277, 768]).
	size mismatch for backbone.layers.0.norm.weight: copying a param with shape torch.Size([2560]) from checkpoint, the shape in current model is torch.Size([768]).
	size mismatch for backbone.layers.0.mixer.A_log: copying a param with shape torch.Size([5120, 16]) from checkpoint, the shape in current model is torch.Size([1536, 16]).
	size mismatch for backbone.layers.0.mixer.D: copying a param with shape torch.Size([5120]) from checkpoint, the shape in current model is torch.Size([1536]).
	size mismatch for backbone.layers.0.mixer.conv1d.weight: copying a param with shape torch.Size([5120, 1, 4]) from checkpoint, the shape in current model is torch.Size([1536, 1, 4]).
	size mismatch for backbone.layers.0.mixer.conv1d.bias: copying a param with shape to

In [ ]:
import torch
from transformers import PhiForCausalLM, AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cpu")

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained("jjhsnail0822/danube-ko-1.8b-base", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained("jjhsnail0822/danube-ko-1.8b-base", trust_remote_code=True)

system_message = "당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요."
prompt = f"[SYSTEM]: {system_message}\n[USER]: {query}\n[context]: {context}\n[ASSISTANT]"

inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_new_tokens=512, max_length=2048)
text = tokenizer.batch_decode(outputs)[0]
print(text) # 14분

c:\Users\USER\anaconda3\envs\vectordb_env\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--jjhsnail0822--danube-ko-1.8b-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Both `max_new_tokens` (=512) and `max_length`(=2048) seem to have been set. `max_new_tokens` wi

[SYSTEM]: 당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요.
[USER]: 자동차 의무보험 미가입에 따른 불이익을 알려줘
[context]: ['보 통 약 관 특 별 약 관 교 통 사 고 발 생 시 대 처 요 령 관 련 법 령 알 기 쉬 운 자 동 차 보 험 이 야 기 개인용자동차보험 9 1 청약내용 확인 관련 보험기간 중 계약 관련 사항이 변경된 경우 계약 담당자나  고객 콜센터(1588-3344, 1600-3434)로 지체없이 연락 주시기 바랍니다. • 고객 주소지 또는 연락처가 변경된 경우 고객님께 발송되는 각종 안내문(만기안내장 등) 수령을 위하여 변경된 주소와 연락처를 알려주시기 바랍니다. • 운전가능자의 범위가 변경된 경우 피보험자동차를 운전하려는 자의 범위 또는 연령이 변경된 경우에는 운전자한정/연령한정 특약을 변경하셔야 합니다. • 새차를 구매하였거나 교체한 경우 새로운 차를 구매하여 기존 차량과 대체한 경우 차량변경 사실을 보험사에 알리고 승인을 받으셔야 합니다. • 새로운 차량 부속품이 추가된 경우 자동차에 장착된 기본 사양 이외에 새로운 부속품이 추가된 경우, 부속품 종류와 가액을 보험회사에 알려주셔야 합니다. 4. 가입자 유의사항 : 보험계약 만료 시 현행 자동차손해배상보장법 제6조 제1항의 규정에 의거, 자동차보험을 가입한 고객님께 보험계약 만료(종료) 전 2회 에 걸쳐 만료사실을 안내하고 있습니다. 구분 안내시기 안내방법 안내대상 1차 안내 만료일 75일~30일전 모바일/이메일/우편 계약자 2차 안내 만료일 30일~10일전 우편 계약자 ※ 보험기간이 1개월 이내인 단기계약의 경우에는 안내가 생략될 수 있습니다. 3. 가입자 유의사항 : 보험계약 변경 시', '보 통 약 관 특 별 약 관 교 통 사 고 발 생 시 대 처 요 령 관 련 법 령 알 기 쉬 운 자 동 차 보 험 이 야 기 개인용자동차보험 25 Q 자기차량손해담보 가입금액은 어떻게 정해지나요? Q 운전 범위에

In [ ]:
import torch
from transformers import PhiForCausalLM, AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cpu")

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained("CarrotAI/Llama-3.2-Rabbit-Ko-3B-Instruct", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained("CarrotAI/Llama-3.2-Rabbit-Ko-3B-Instruct", trust_remote_code=True)

system_message = "당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요."
prompt = f"[SYSTEM]: {system_message}\n[USER]: {query}\n[context]: {context}\n[ASSISTANT]"

inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_new_tokens=512, max_length=2048)
text = tokenizer.batch_decode(outputs)[0]
print(text) # 17분

c:\Users\USER\anaconda3\envs\vectordb_env\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--CarrotAI--Llama-3.2-Rabbit-Ko-3B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.75it/s]
The attention 

<|begin_of_text|>[SYSTEM]: 당신은 보험 상담 ASSISTANT입니다. 보험 약관인 context 기반으로 query에 대해 답변해 주세요.
[USER]: 자동차 의무보험 미가입에 따른 불이익을 알려줘
[context]: ['보 통 약 관 특 별 약 관 교 통 사 고 발 생 시 대 처 요 령 관 련 법 령 알 기 쉬 운 자 동 차 보 험 이 야 기 개인용자동차보험 9 1 청약내용 확인 관련 보험기간 중 계약 관련 사항이 변경된 경우 계약 담당자나  고객 콜센터(1588-3344, 1600-3434)로 지체없이 연락 주시기 바랍니다. • 고객 주소지 또는 연락처가 변경된 경우 고객님께 발송되는 각종 안내문(만기안내장 등) 수령을 위하여 변경된 주소와 연락처를 알려주시기 바랍니다. • 운전가능자의 범위가 변경된 경우 피보험자동차를 운전하려는 자의 범위 또는 연령이 변경된 경우에는 운전자한정/연령한정 특약을 변경하셔야 합니다. • 새차를 구매하였거나 교체한 경우 새로운 차를 구매하여 기존 차량과 대체한 경우 차량변경 사실을 보험사에 알리고 승인을 받으셔야 합니다. • 새로운 차량 부속품이 추가된 경우 자동차에 장착된 기본 사양 이외에 새로운 부속품이 추가된 경우, 부속품 종류와 가액을 보험회사에 알려주셔야 합니다. 4. 가입자 유의사항 : 보험계약 만료 시 현행 자동차손해배상보장법 제6조 제1항의 규정에 의거, 자동차보험을 가입한 고객님께 보험계약 만료(종료) 전 2회 에 걸쳐 만료사실을 안내하고 있습니다. 구분 안내시기 안내방법 안내대상 1차 안내 만료일 75일~30일전 모바일/이메일/우편 계약자 2차 안내 만료일 30일~10일전 우편 계약자 ※ 보험기간이 1개월 이내인 단기계약의 경우에는 안내가 생략될 수 있습니다. 3. 가입자 유의사항 : 보험계약 변경 시', '보 통 약 관 특 별 약 관 교 통 사 고 발 생 시 대 처 요 령 관 련 법 령 알 기 쉬 운 자 동 차 보 험 이 야 기 개인용자동차보험 25 Q 자기차량손해담보 가입금액은 어떻